In [ ]:
from flask import Flask, request, jsonify
from functools import wraps
import logging

In [ ]:
# Load the model for inference
model = joblib.load('fraud_detection_model.pkl')

# Initialize Flask app
app = Flask(__name__)

# Authentication decorator
def authenticate(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        auth = request.headers.get("Authorization")
        if not auth or auth != "Bearer secure_token":
            return jsonify({"message": "Unauthorized"}), 401
        return f(*args, **kwargs)
    return decorated_function

@app.route('/predict', methods=['POST'])
@authenticate
def predict():
    data = request.get_json()
    features = np.array([data['amount'], data['time_of_day'], data['transaction_frequency'], data['location_change']]).reshape(1, -1)
    prediction = model.predict(features)[0]
    logging.info(f"Transaction predicted: {data}, Fraud: {prediction}")
    
    if prediction == 1:
        logging.warning(f"Fraudulent transaction detected: {data}")
    
    return jsonify({'fraud_prediction': int(prediction)})

if __name__ == '__main__':
    app.run(debug=True)